# Plotting species distribution areas (IUCN spatial data)

## Imports

### Libraries

In [1]:
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt

%matplotlib widget

### Data

In [2]:
shp = gpd.read_file('/Users/bell/OneDrive - Instituto Politecnico Nacional/Rscripts/Mapas/MARINE_MAMMALS/MARINE_MAMMALS.shp')
#shore = gpd.read_file('/Users/bell/OneDrive - Instituto Politecnico Nacional/Rscripts/Mapas/gshhg-shp/GSHHS_shp/i/GSHHS_i_L1.shp')
#shp.head()

## Custom Functions

In [3]:
def RGB(rgb_code):
    """Transforms 0-255 rgb colors to 0-1 scale"""
    col = tuple(x/255 for x in rgb_code)
    return col

In [4]:
def get_ranges(species, shp, drop_columns = None):
    """Returns a list of simplifyied distribution data from a shape downloaded from the IUCN. 
       Arguments:
       species: a list of binomial species names to get the distributions from
       shp: the shapefile variable containing the distribution ranges
       drop_columns: Columns with attributes to drop from the original shapefile, if None, no columns are dropped"""
    #ranges = list()
    if drop_columns == None:
        ranges = [shp[shp.binomial == sp] for sp in species]
        #for i,sp in set(enumerate(species)):
        #    ranges.append(shp[shp.binomial == sp])
    else:
        #shp = shp.drop(columns = drop_columns)
        ranges = [shp[shp.binomial == sp].drop(columns = drop_columns) for sp in species]
        #for i,sp in set(enumerate(species)):
        #    ranges.append(shp[shp.binomial == sp])
        
    
    return ranges

In [5]:
def plot_dist_ranges(ranges, colors, shore = None, extent = None):
    """Plots the distribution ranges of the species contained in the ranges variable.
        Arguments:
        ranges: A list in which each element corresponds to a distribution range (geometry) and attributes from distribution data downloadaded from the IUCN
        colors: A list of colors for each species. If None, defaults to the 'cool' colormap.
        shore: Variable storing the coastline
        extent: A list with the limits of the plot, in the form: [x_inf, x_sup, y_inf, y_sup]"""
    font = {'family': 'Montserrat',
            'weight': 500}
    plt.rcParams['font.family'] = font['family']
    plt.rcParams['font.weight'] = font['weight']
    #plt.rcParams[]
    #font = 'Montserrat-Regular'
    #color = (27/255,109/255,183/255)
    fig = plt.figure(figsize = (8,5))
    ax = plt.gca()

    for i,r in list(enumerate(ranges)):
        r.plot(alpha = 0.35, ax = ax, color = colors[i])
    if shore == None:
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        world.plot(color = 'lightslategray', alpha = 0.5, ax = ax)
    else:
        shore.plot(color = 'lightslategray', alpha = 0.5, ax = ax)
    plt.tight_layout()
    for spine in ax.spines.values():
        spine.set_visible(False)

    ax.tick_params(axis        = 'both',
                   which       = 'both',
                   bottom      = False,
                   top         = False,
                   left        = False,
                   labelleft   = True,
                   labelbottom = True)
    if extent == None:
        extent = [-180, 180, -90, 90]
        plt.axis(extent)
        plt.yticks(np.arange(-80,120,40))
    else:
        long_range = abs(extent[0] - extent[1])/4
        plt.yticks(np.linspace(extent[0], extent[1], 4))
        
    plt.xlabel('Longitud (º)')
    plt.ylabel('Latitud (º)')
    plt.labelweight = font['weight']

## Extracting and plotting the data

In [6]:
species = ['Stenella attenuata', 'Stenella longirostris']
drop_columns = ['id_no','presence', 'origin', 'source', 'seasonal', 'compiler', 'yrcompiled',
                'citation', 'dist_comm', 'island', 'subspecies', 'subpop', 'tax_comm', 'kingdom',
                'phylum', 'class', 'order_', 'family', 'genus', 'category', 'marine', 'terrestial', 'freshwater']
ranges = get_ranges(species, shp, drop_columns)

In [7]:
#colors = ['gold', (27/255,109/255,183/255)]
colors = [RGB((121,227,249)), RGB((27,109,183)), 'firebrick']
#extent = [-116,-112, 26, 33]
plot_dist_ranges(ranges, colors);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
plt.savefig('DistSASL.jpg', bbox_inches = 'tight', pad_inches = 0.1, transparent = False, dpi = 300)